## Text classification

In [1]:
import pandas as pd
import numpy as np

In [22]:
# read json 
import json

# json file paths
f = open(r"final caption model\christine_scam\scam_0_500.json")

data = json.load(f)

filename = list(data.keys())
caption = list(data.values())

df = pd.DataFrame(data={'filename':filename, 'caption':caption})
df.head()


,filename,caption
0,nathangilbert1301.jpg,man in black shirt is standing in front of wom...
1,keneth1.jpg,man in black shirt is standing in front of she...
2,Jhjh281.jpg,man in black shirt is standing in front of wom...
3,OlegWilliams1.jpg,man in black shirt is standing in front of crowd
4,brucep1.jpg,man in black shirt is sitting on bench with he...


In [2]:
pip install -U imbalanced-learn

     -------------------------------------- 199.3/199.3 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pingouin 0.5.2 requires scikit-learn<1.1.0, but you have scikit-learn 1.1.3 which is incompatible.


In [2]:
# Oversampling and under sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

In [3]:
combined_df = pd.read_csv("combined_profile.csv")

In [49]:
pd.set_option("max_colwidth", None)

In [50]:
combined_df.head(10)


,username,age,occupation,status,gender,filtered sentence,y
0,123canwe,66.0,Retired,single,male,I full fire friskier hell,0
1,123WILFREDO,28.0,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
2,1907,48.0,Construction,single,male,laid back earth good sense humor,0
3,52Jim52,70.0,Retired,divorced,male,Retired owner aerospace consuloting firm,0
4,Aaron90,28.0,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im really fun person I like camping bone fire etc,0
5,abdelghani,71.0,retired,divorced,male,problem free flexibleseek love happiness,0
6,Abdul99,54.0,Executive,separated,male,Easy going Highly educated Life beautiful Travels,0
7,abdulrahmanwaly,33.0,journalist,single,male,name Abdulrahman person ethics religion work field journalism live Cairo,0
8,Abou,33.0,Designer,single,male,I serious honest trustworthy like share love fan culture new different person,0
9,Accydave,73.0,Retired,divorced,male,My name David Im 71 years old divorced retired 3 grownup children son 2 daughters,0


In [66]:
# features
X = combined_df['filtered sentence']

# target
y = combined_df['y']

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 424)

# output for slides

In [10]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

In [58]:
XX = combined_df['filtered sentence'].iloc[2:5]

In [60]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(XX)
xtrain_tfidf =  tfidf_vect.transform(XX)


In [63]:
print(tfidf_vect.get_feature_names())  
print('\n')
print(xtrain_tfidf.shape)
print('\n')
print(xtrain_tfidf)

['2', 'aaron', 'aerospace', 'back', 'bone', 'camping', 'consuloting', 'earth', 'etc', 'fire', 'firm', 'fun', 'good', 'hello', 'humor', 'i', 'im', 'laid', 'like', 'owner', 'person', 'really', 'retired', 'sense', 'vermont', 'years']


(3, 26)


  (0, 23)	0.4082482904638631
  (0, 17)	0.4082482904638631
  (0, 14)	0.4082482904638631
  (0, 12)	0.4082482904638631
  (0, 7)	0.4082482904638631
  (0, 3)	0.4082482904638631
  (1, 22)	0.4472135954999579
  (1, 19)	0.4472135954999579
  (1, 10)	0.4472135954999579
  (1, 6)	0.4472135954999579
  (1, 2)	0.4472135954999579
  (2, 25)	0.21821789023599233
  (2, 24)	0.43643578047198467
  (2, 21)	0.21821789023599233
  (2, 20)	0.21821789023599233
  (2, 18)	0.21821789023599233
  (2, 16)	0.21821789023599233
  (2, 15)	0.43643578047198467
  (2, 13)	0.21821789023599233
  (2, 11)	0.21821789023599233
  (2, 9)	0.21821789023599233
  (2, 8)	0.21821789023599233
  (2, 5)	0.21821789023599233
  (2, 4)	0.21821789023599233
  (2, 1)	0.21821789023599233
  (2, 0)	0.2182178902359923

In [65]:
print(xtrain_tfidf.toarray())


[[0.         0.         0.         0.40824829 0.         0.
  0.         0.40824829 0.         0.         0.         0.
  0.40824829 0.         0.40824829 0.         0.         0.40824829
  0.         0.         0.         0.         0.         0.40824829
  0.         0.        ]
 [0.         0.         0.4472136  0.         0.         0.
  0.4472136  0.         0.         0.         0.4472136  0.
  0.         0.         0.         0.         0.         0.
  0.         0.4472136  0.         0.         0.4472136  0.
  0.         0.        ]
 [0.21821789 0.21821789 0.         0.         0.21821789 0.21821789
  0.         0.         0.21821789 0.21821789 0.         0.21821789
  0.         0.21821789 0.         0.43643578 0.21821789 0.
  0.21821789 0.         0.21821789 0.21821789 0.         0.
  0.43643578 0.21821789]]


# Naive Bayes classifier
- Taken from: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [68]:
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#WITHOUT OVERSAMPLING/UNDER SAMPLING
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
text_clf = text_clf.fit(X_train, y_train)

In [12]:
predictions = text_clf.predict(X_test)

### Evaluating the NB classifier

In [13]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("original dataset")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

original dataset
Accuracy: 0.9262295081967213
Precision: 0.8294993234100135
recall: 1.0
f1_score: 0.9068047337278106


In [69]:
#random oversampling
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

text_clf = MultinomialNB()
ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf = text_clf.fit(X_train_ros, y_train_ros)
predictions = text_clf.predict(xvalid_tfidf)

In [70]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.9268149882903981
Precision: 0.8306233062330624
recall: 1.0
f1_score: 0.9074759437453738


In [71]:
#random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf = MultinomialNB()
text_clf = text_clf.fit(X_train_rus, y_train_rus)
predictions = text_clf.predict(xvalid_tfidf)

In [72]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.9256440281030445
Precision: 0.8283783783783784
recall: 1.0
f1_score: 0.9061345158906134


# SVM classifier

In [73]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42)),
])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

In [74]:
predictions = text_clf_svm.predict(X_test)

### Evaluating the SVM classifier

In [75]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.9227166276346604
Precision: 0.8228187919463087
recall: 1.0
f1_score: 0.9027982326951398


In [76]:
#random oversampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_ros, y_train_ros)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [77]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

oversampling
Accuracy: 0.9221311475409836
Precision: 0.82171581769437
recall: 1.0
f1_score: 0.9021339220014717


In [78]:
#random undersampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_rus, y_train_rus)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [79]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

undersampling
Accuracy: 0.9221311475409836
Precision: 0.82171581769437
recall: 1.0
f1_score: 0.9021339220014717


# Logistic regression

In [86]:
from sklearn.linear_model import LogisticRegression
# Train model
clf_lg = LogisticRegression()  
clf_lg = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf',LogisticRegression()),
])
clf_lg = clf_lg.fit(X_train, y_train) 

In [81]:
lg_predictions = clf_lg.predict(X_test)

### Evaluating logistic regression classifier

In [71]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.9106641721234799
Precision: 0.7995283018867925
recall: 0.9699570815450643
f1_score: 0.8765352294764058


In [87]:
#random oversampling
clf_lg = LogisticRegression()  

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_ros, y_train_ros)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [88]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

oversampling
Accuracy: 0.9320843091334895
Precision: 0.8446601941747572
recall: 0.9934747145187602
f1_score: 0.9130434782608695


In [89]:
#random undersampling
clf_lg = LogisticRegression()  

ros = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_rus, y_train_rus)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [90]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

undersampling
Accuracy: 0.9250585480093677
Precision: 0.8272604588394062
recall: 1.0
f1_score: 0.9054652880354505


# Reading real profile

In [2]:
import json
import os
import glob
import pandas as pd

json_dir = 'data/realprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
realdf = pd.concat(dfs)

In [5]:
realdf.head()

,gender,age,location,status,username,ethnicity,occupation,description,match_age,children,orientation,religion,smoking,drinking,intent,site
0,female,62 y.o.,"Nashville, TN, USA",divorced,Suzy60,white,Teacher,-,from 45 to 65,1-2 living with me,Straight,Spiritual,social smoker,occasional drinker,Friendship,realSuzy60.json
0,male,56 y.o.,"Pedregal del Cortez, 23018 La Paz, B.C.S., México",divorced,cone66,hispanic,construcción,-,from 38 to 49,1-2 living elsewhere,Straight,Christian,non-smoker,social drinker,"Fun, Friendship, Romance",realcone66.json
0,male,53 y.o.,"Stockholm, Sweden",separated,DrOo,white,teacher,-,from 30 to 48,1-2 living with me,Straight,Christian,non-smoker,social drinker,"Romance, Serious Relationship, Marriage",realDrOo.json
0,female,40 y.o.,"Bogotá, Colombia",single,Leyjo,hispanic,Fisioterapeuta,-,from 31 to 45,no children,Straight,Christian,non-smoker,social drinker,Serious Relationship,realLeyjo.json
0,female,31 y.o.,"Manizales, Caldas, Colombia",single,Paoramirez08,white,-,-,from 23 to 40,no children,Straight,Other,non-smoker,never,"Romance, Serious Relationship, Marriage",realPaoramirez08.json


# Reading scam profile

In [ ]:
json_dir = 'data/scamprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
scamdf = pd.concat(dfs)

# Reading image caption json (real)

In [6]:
json_dir = 'final caption model/xinyi_real'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
image_real_df = pd.concat(dfs)

In [7]:
image_real_df.head()

,123canwe.jpg,123WILFREDO.jpg,52Jim52.jpg,Aaron90.jpg,Abou.jpg,Agatha123.jpg,Ahin.jpg,Airik.jpg,alaaeldin.jpg,Alexmmar.jpg,...,zJoseManueL.jpg,Zohe.jpg,zorro.jpg,Zschweitz418.jpg,Zulemaa.jpg,Zulia.jpg,ZullynAzul2.jpg,Zwadiemarie22.jpg,Zxxxx.jpg,_Diana_.jpg
0,man in red shirt is standing on bench next to ...,man in red shirt is sitting on the edge of river,man in black shirt is standing in front of woo...,two girls are playing in the grass,man in red shirt and black shorts is walking d...,young girl in pink shirt is standing in front ...,man in black shirt is standing in front of store,man in black shirt is smiling at the camera,man in black shirt and black shirt is standing...,man in black shirt is smiling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,man in red shirt is standing on bench next to ...,man in red shirt is sitting on the edge of river,man in black shirt is standing in front of woo...,two girls are playing in the grass,man in red shirt and black shorts is walking d...,young girl in pink shirt is standing in front ...,man in black shirt is standing in front of store,man in black shirt is smiling at the camera,man in black shirt and black shirt is standing...,man in black shirt is smiling,...,two girls are playing in the grass,woman in black shirt and sunglasses is sitting...,two girls are playing in the grass,man in red shirt is standing in front of crowd,two girls are playing in the grass,two women are sitting on bench in front of the...,two girls are playing in the grass,two girls are playing in the grass,two girls are sitting on the floor,two girls are playing in the grass
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
